# Robust SVM (WORK IN PROGRESS)

Consider the problem of learning a support vector machine (SVM) from a set of m data points {(xi , yi )}mi=1 . Sup- pose we would like to (additionally) model the fact that our data collection process is noisy (in order to gain ro- bustness in our solution), by incorporating the belief that (say) xi ∼ Normal(μ1,Σ1) for all i where yi = 1 and xi ∼ Normal(μ2,Σ2) for all i where yi = −1 into the learning process. We can thereby pose the following chance-constrained variant of the canonical (soft-margin)

The optimization problem becomes:
$$
\begin{aligned}
& \min_{w, b, \xi_i} \frac{1}{2} \| w \|^2 + C \sum_{i=1}^m \xi_i \\
& \text{subject to:} \\
& \forall i, \forall (x_i, y_i) \in \mathcal{S}_i: \\
& \quad y_i (w^T x_i + b) \geq 1 - \xi_i, \\
& \quad \xi_i \geq 0
\end{aligned}
$$

In [1]:
import numpy as np
import cvxpy as cp
import lropt


In [2]:
m = 100  # Number of data points
C = 1.0  # Regularization parameter
sigma = cp.Variable(m, nonneg = True) # Probability threshold
eta = 0.95
mean = 0
std_dev = 1
num_samples = 1000

In [3]:
np.random.seed(1)
w = cp.Variable(m)
b = cp.Variable()

ux = lropt.UncertainParameter(m, uncertainty_set = lropt.Ellipsoidal())
uy = lropt.UncertainParameter(m, uncertainty_set = lropt.Ellipsoidal(), boolean = True)
y = 2 * (np.random.rand(m) < 0.5) - 1  # Random labels (either 1 or -1)

In [4]:
objective = cp.Minimize(cp.norm2(w) + C*(cp.sum(sigma)))

In [5]:
mu_1 = 3.0
sigma_1 = 2.0
xi = np.random.normal(mu_1, sigma_1, m)

In [6]:
w @ (xi + ux) + b

Expression(AFFINE, UNKNOWN, ())

In [7]:
constraints = []
for i in range(m):
    constraint = w[i]*(xi[i] + ux[i]) + b >= 1 - sigma[i]
    constraints.append(constraint)
constraints   

[Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNKNOWN, ())),
 Inequality(Expression(AFFINE, UNK

In [8]:
prob = lropt.RobustProblem(objective, constraints)
prob.solve()

/Users/mj5676/Documents/Project2/lropt_v1/lib/python3.12/site-packages/scipy/sparse/_index.py:134: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)
/Users/mj5676/Documents/Project2/lropt_v1/lib/python3.12/site-packages/cvxpy/utilities/torch_utils.py:61: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:623.)
  return torch.sparse.FloatTensor(i, v, torch.Size(value_coo.shape)).to(dtype)


In [9]:
print(w.value)
print(sigma.value)

[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [10]:
sigma = cp.Variable(nonneg = True)
constraints = [w @ (xi + ux) + b >= 1 - sigma]
objective = cp.Minimize(cp.norm2(w) + C*(cp.sum(sigma)))
prob = lropt.RobustProblem(objective, constraints)
prob.solve()

In [11]:
w.value

array([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0.])

In [12]:
sigma.value

array(-0.)